<a href="https://colab.research.google.com/github/ymuto0302/PublicLecture2025/blob/main/pref_crime_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 都道府県セグメンテーション

県民所得，人口密度，犯罪率のデータに基づき，都道府県のグループ分けを試みる。

本ノートブックにて用いるデータは，下記の公的データをまとめたものである。
- 県民所得データ：令和４年度県民経済計算（内閣府経済社会総合研究所）
- 人口密度データ：uub.jp（都道府県市区町村統計サイト）から令和５年10月1日現在の推計人口および令和５年7月1日現在の国土地理院データから面積を取得
- 人口あたり犯罪件数：令和5年犯罪統計資料（警察庁），令和6年警察白書 → 人口10万人当たり刑法犯認知件数（令和5年）

In [ ]:
# グラフ表示の際，日本語を表示できるように japanize_matplotlib をインストール
!pip install japanize_matplotlib

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

## STEP 1: データの読み込みと確認
独自に作成した crime.csv を読み込む。

In [ ]:
# データの読み込み
# Mall Customer Segmentation Datasetを使用
df = pd.read_csv('crime.csv')

print(f"データ件数: {df.shape[0]}")

# データの先頭5行を確認
df.head()

In [ ]:
print("\n=== 基本統計量の確認 ===")
df.describe()

In [ ]:
print("\n=== 欠損値の確認 ===")
df.isnull().sum()

In [ ]:
# データ分布の可視化

# クラスタリング用の特徴量を選択: 県民所得と犯罪件数を使用
features = ['県民所得', '犯罪件数']
X = df[features].copy()

# 散布図プロット
plt.scatter(X['県民所得'], X['犯罪件数'], alpha=0.7)
plt.xlabel('県民所得')
plt.ylabel('犯罪件数')
plt.title('散布図')
plt.show()

## STEP 2: データの前処理（標準化）

In [ ]:
# データの標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n=== 標準化後のデータ統計 ===")
X_scaled_df = pd.DataFrame(X_scaled)
X_scaled_df.describe()

In [ ]:
from sklearn.metrics import silhouette_score

# 最適なクラスター数の決定

# エルボー法 ＆ シルエット分析
inertias = []
silhouette_scores = []
k_range = range(2, 11) # クラスタ数の候補値を 2〜10 とする

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# 結果の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(k_range, inertias, 'bo-')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, 'ro-')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.grid(True)

plt.tight_layout()
plt.show()

# 最適なクラスター数を出力
best_k_silhouette = k_range[np.argmax(silhouette_scores)]
print(f"シルエット分析による最適クラスタ数: {best_k_silhouette}")

## STEP 3: k-means法によるクラスタリング

In [ ]:
# k=6でクラスタリング実行
optimal_k = 6
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)

# 結果をデータフレームに追加
df['Cluster'] = cluster_labels

# クラスター別の件数
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
print(f"\n各クラスターの件数:")
for i, count in enumerate(cluster_counts):
    print(f"クラスター {i}: {count}件")

## STEP 4: 結果の評価と解釈

In [ ]:
# クラスター別の統計情報
print("\n=== クラスター別統計情報 ===")
cluster_summary = df.groupby('Cluster')[['県民所得', '人口密度', '犯罪件数']].agg({
    '県民所得': ['mean', 'std'],
    '人口密度': ['mean', 'std'],
    '犯罪件数': ['mean', 'std']
}).round(2)

print(cluster_summary)

In [ ]:
# クラスタリング結果の散布図
plt.scatter(X[features[0]],  # X
            X[features[1]], # Y
            c=df['Cluster'], # 色はクラスタ番号に従う
            alpha=0.7) # バブルの透明度

plt.xlabel('県民所得')
plt.ylabel('犯罪件数')
plt.title('都道府県セグメンテーションの結果')
plt.grid(True, alpha=0.3)

In [ ]:
# 各クラスタに含まれる都道府県名を観察する
for i in range(optimal_k):
    prefs = df[df['Cluster']==i]['都道府県'].tolist()
    print(f"クラスタ {i}: {prefs}")

---
## 人口密度 vs 犯罪件数

## STEP 1: データの確認

In [ ]:
# データ分布の可視化

# クラスタリング用の特徴量を選択: 人口密度と犯罪件数を使用
features = ['人口密度', '犯罪件数']
X = df[features].copy()

# scatter plot
plt.scatter(X['人口密度'], X['犯罪件数'], alpha=0.7)
plt.xlabel('人口密度')
plt.ylabel('犯罪件数')
plt.title('散布図')
plt.show()


## STEP 2: データの前処理（対数変換 & 標準化）

東京都と大阪府の人口密度が飛び抜けてしまうため，横軸を対数スケールに変換して表示する

In [ ]:
# データ分布の可視化： 人口密度を対数スケールへ変換
plt.scatter(X['人口密度'], X['犯罪件数'], alpha=0.7)
plt.xlabel('人口密度(対数)')
plt.ylabel('犯罪件数')
plt.title('散布図')
plt.xscale('log') # 横軸を対数スケールにて表示
plt.show()


In [ ]:
# 人口密度を対数スケールに変換した後，標準化を行う
df['log_人口密度'] = np.log(df['人口密度'])
features = ['log_人口密度', '犯罪件数']
X = df[features].copy()

# データの標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n=== 標準化後のデータ統計 ===")
X_scaled_df = pd.DataFrame(X_scaled)
X_scaled_df.describe()

In [ ]:
from sklearn.metrics import silhouette_score

# 最適なクラスター数の決定

# エルボー法 ＆ シルエット分析
inertias = []
silhouette_scores = []
k_range = range(2, 11) # クラスタ数の候補値を 2〜10 とする

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# 結果の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(k_range, inertias, 'bo-')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, 'ro-')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.grid(True)

plt.tight_layout()
plt.show()

# 最適なクラスター数を出力
best_k_silhouette = k_range[np.argmax(silhouette_scores)]
print(f"シルエット分析による最適クラスタ数: {best_k_silhouette}")

## STEP 3: k-means法によるクラスタリング

In [ ]:
# k=5でクラスタリング実行
optimal_k = 5
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)

# 結果をデータフレームに追加
df['Cluster'] = cluster_labels

# クラスター別の件数
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
print(f"\n各クラスターの件数:")
for i, count in enumerate(cluster_counts):
    print(f"クラスター {i}: {count}件")

## STEP 4: 結果の評価と解釈

In [ ]:
# クラスター別の統計情報
print("\n=== クラスター別統計情報 ===")
cluster_summary = df.groupby('Cluster')[['県民所得', '人口密度', '犯罪件数']].agg({
    '県民所得': ['mean', 'std'],
    '人口密度': ['mean', 'std'],
    '犯罪件数': ['mean', 'std']
}).round(2)

print(cluster_summary)

In [ ]:
# クラスタリング結果の散布図
plt.scatter(X[features[0]],  # X
            X[features[1]], # Y
            c=df['Cluster'], # 色はクラスタ番号に従う
            alpha=0.7) # バブルの透明度

plt.xlabel('人口密度(対数)')
plt.ylabel('犯罪件数')
plt.title('都道府県セグメンテーションの結果')
plt.grid(True, alpha=0.3)

In [ ]:
# 各クラスタに含まれる都道府県名を観察する
for i in range(optimal_k):
    prefs = df[df['Cluster']==i]['都道府県'].tolist()
    print(f"クラスタ {i}: {prefs}")